In [1]:
%matplotlib inline
from parcels import FieldSet, Field, ParticleSet, AdvectionRK4, JITParticle, ErrorCode, VectorField, Variable
from datetime import timedelta as delta
from glob import glob
import numpy as np
import xarray as xr

In [2]:
ddir = '/science/projects/oceanparcels/SKIMulator/V4.0/Global_l2d/'
files = sorted(glob(ddir+'WW3_GL_metop_2019_6b_A__l2d*.nc'))

dimensions = {'lon': 'lon', 'lat': 'lat', 'time': 'time'}
variables = {}
variables['no_err'] = {'U': 'ux_noerr', 'V': 'uy_noerr'}
variables['obs'] = {'U': 'ux_obs', 'V': 'uy_obs'}

fsets = ['no_err', 'obs']
fieldset = {}
for f in fsets:
    fieldset[f] = FieldSet.from_netcdf(files, variables[f], dimensions) 

    fieldset[f].computeTimeChunk(0, 1)

In [3]:
variables = {'U_unbeach': 'unBeachU', 'V_unbeach': 'unBeachV', 'Mask': 'Mask'}
dimensions = {'lon': 'lon', 'lat': 'lat'}
fieldsetUnBeach = FieldSet.from_netcdf('skimulator_unbeaching_vel.nc', variables, dimensions)

for f in fsets:
    fieldset[f].add_field(fieldsetUnBeach.U_unbeach)
    fieldset[f].add_field(fieldsetUnBeach.V_unbeach)
    fieldset[f].add_field(fieldsetUnBeach.Mask)

    UVunbeach = VectorField('UVunbeach', fieldset[f].U_unbeach, fieldset[f].V_unbeach)
    fieldset[f].add_vector_field(UVunbeach)

In [4]:
def UnBeaching(particle, fieldset, time):
    m = fieldset.Mask[0, particle.depth, particle.lat, particle.lon]
    if m < 0.99:
        (ub, vb) = fieldset.UVunbeach[time, particle.depth, particle.lat, particle.lon]
        particle.lon += ub * particle.dt
        particle.lat += vb * particle.dt

def DeleteParticle(particle, fieldset, time):
    particle.delete()

In [6]:
for f in fsets:
    fieldset[f].check_complete()

lons = []
lats = []
for lon in np.arange(360-80, 350, 0.5):
    for lat in np.arange(10, 50, 0.5):
        if fieldset['no_err'].Mask[0, 0, lat, lon] > 0.99:
            lons.append(lon)
            lats.append(lat)
print('%d particle locations' % len(lons))

10317 particle locations


In [7]:
for f in fsets:
    fname = "/scratch/evansebille/skimulator_%s.nc" % f
    print(fname)

    pset = ParticleSet(fieldset=fieldset[f], pclass=JITParticle, lon=lons, lat=lats)

    outfile = pset.ParticleFile(name=fname, outputdt=delta(days=5))

    pset.execute(AdvectionRK4+pset.Kernel(UnBeaching), dt=delta(hours=1), output_file=outfile,
                 recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})
    outfile.close()

/scratch/evansebille/skimulator_no_err.nc


INFO: Compiled JITParticleAdvectionRK4UnBeaching ==> /tmp/parcels-24236/189064886c94ab33638e433da881bd52.so
INFO: Temporary output files are stored in /scratch/evansebille/out-RENNQXVE.
INFO: You can use "parcels_convert_npydir_to_netcdf /scratch/evansebille/out-RENNQXVE" to convert these to a NetCDF file during the run.
100% (2851200.0 of 2851200.0) |##########| Elapsed Time: 0:00:58 Time:  0:00:58


/scratch/evansebille/skimulator_obs.nc


INFO: Compiled JITParticleAdvectionRK4UnBeaching ==> /tmp/parcels-24236/453e592b0701ab49bcde4d8d4356e2c8.so
INFO: Temporary output files are stored in /scratch/evansebille/out-QRKZRTHQ.
INFO: You can use "parcels_convert_npydir_to_netcdf /scratch/evansebille/out-QRKZRTHQ" to convert these to a NetCDF file during the run.
100% (2851200.0 of 2851200.0) |##########| Elapsed Time: 0:00:57 Time:  0:00:57
